In [2]:
import digitalhub as dh
import pandas as pd
import requests
import os

In [3]:
PROJECT = "llm"
project = dh.get_or_create_project(PROJECT)


In [10]:
%%writefile "src/train_model.py"

from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer
import numpy as np
import evaluate
import os

from digitalhub_runtime_python import handler

@handler()
def train(project):
    tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
    metric = evaluate.load("accuracy")
    
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)
    
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)
    
    
    dataset = load_dataset("yelp_review_full")
    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    
    model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)
    
    training_args = TrainingArguments(output_dir="test_trainer")
    
    small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
    small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
    
    training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=small_train_dataset,
        eval_dataset=small_eval_dataset,
        compute_metrics=compute_metrics,
    )
    
    trainer.train()

    save_dir = "model"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    trainer.save_model(save_dir)
    tokenizer.save_pretrained(save_dir)


    project.log_model(
            name="test_llm_model", 
            kind="huggingface", 
            source=save_dir
    )    

Overwriting src/train_model.py


In [11]:
train_func = project.new_function(
                         name="train_model",
                         kind="python",
                         python_version="PYTHON3_9",
                         code_src="src/train_model.py",
                         handler="train",
                         requirements=["evaluate", "transformers[torch]", "torch", "torchvision", "accelerate"]
                        )

In [12]:
train_run=train_func.run(action="job", 
                         local_execution=False,
                         profile="template-a100"
                        )

In [13]:

llm_function = project.new_function("llm_classification", 
                                   kind="huggingfaceserve",
                                   model_name="bertmodel",
                                   path="s3://datalake/llm/model/test_llm_model/9e53f1f2-8ae9-418a-9af3-1c5f64b19931/"
                                  )


In [14]:
llm_run = llm_function.run(action="serve", profile="template-a100")

In [ ]:
SERVICE_URL = llm_run.refresh().status.to_dict()["service"]["url"]
MODEL_NAME = "bertmodel"

with requests.post(f'http://{SERVICE_URL}/v2/models/{MODEL_NAME}/infer', json={
    "inputs": [
        {
        "name": "input-0",
        "shape": [2],
        "datatype": "BYTES",
        "data": ["Hello, my dog is cute", "I am feeling sad"]
        }
    ]
}) as r:
    res = r.json()
print(res)
